In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip "./drive/MyDrive/multicampus/cats_and_dogs.zip" -d "/content"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/__MACOSX/cats_and_dogs/train/cats/._cat_1862.jpg  
  inflating: /content/cats_and_dogs/train/cats/cat_3913.jpg  
  inflating: /content/__MACOSX/cats_and_dogs/train/cats/._cat_3913.jpg  
  inflating: /content/cats_and_dogs/train/cats/cat_3907.jpg  
  inflating: /content/__MACOSX/cats_and_dogs/train/cats/._cat_3907.jpg  
  inflating: /content/cats_and_dogs/train/cats/cat_1876.jpg  
  inflating: /content/__MACOSX/cats_and_dogs/train/cats/._cat_1876.jpg  
  inflating: /content/cats_and_dogs/train/cats/cat_0598.jpg  
  inflating: /content/__MACOSX/cats_and_dogs/train/cats/._cat_0598.jpg  
  inflating: /content/cats_and_dogs/train/cats/cat_1686.jpg  
  inflating: /content/__MACOSX/cats_and_dogs/train/cats/._cat_1686.jpg  
  inflating: /content/cats_and_dogs/train/cats/cat_3091.jpg  
  inflating: /content/__MACOSX/cats_and_dogs/train/cats/._cat_3091.jpg  
  inflating: /content/cats_and_dogs/train/cats/cat_3085.jpg  
  inflating: /conte

In [ ]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=4f6aa456ec6b7d0b7074e77753f96f3ecc8b841b28bbd14e4075ba1502ab9b8b
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### 데이터셋 준비

In [4]:
from torch.utils.data import Dataset
import pandas as pd
from torchvision.io import read_image

In [5]:
class CustomImageDataset(Dataset):
  def __init__(self, image_dir, transform=None):
    self.image_dir = image_dir
    self.img_labels = pd.read_csv(f"{image_dir}/label.csv")
    self.transform = transform

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    image_path = f"{self.image_dir}/{self.img_labels.iloc[idx, 0]}"
    image = read_image(image_path)
    if self.transform:
      image = self.transform(image)
    label = self.img_labels.iloc[idx, 1]
    return image, label

In [6]:
# from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

# weights = EfficientNet_B0_Weights.IMAGENET1K_V1
# transform = weights.transforms()

### dataset, 데이터 로더 생성

In [7]:
train_dataset = CustomImageDataset("./cats_and_dogs/train", transform=transform)
val_dataset = CustomImageDataset("./cats_and_dogs/test", transform=transform)

In [8]:
from torch.utils.data import DataLoader

batch_size = 128
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
train_dataset[0][0].numpy()

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


array([[[-1.1075436 , -0.6622999 , -1.2274169 , ..., -0.7650484 ,
         -0.9020464 , -1.1589178 ],
        [-1.3301654 , -0.9020464 , -1.2274169 , ..., -0.78217316,
         -0.6622999 , -1.0219197 ],
        [-1.5870366 , -1.3130406 , -1.3644148 , ..., -0.8335474 ,
         -0.9020464 , -1.0390445 ],
        ...,
        [-1.9295317 , -1.9124069 , -1.8781574 , ..., -0.7650484 ,
         -0.67942464, -0.69654936],
        [-1.8781574 , -1.8610327 , -1.8610327 , ..., -0.5424266 ,
         -0.5081771 , -0.6109256 ],
        [-1.7582842 , -1.7754089 , -1.8096584 , ..., -0.5253019 ,
         -0.6109256 , -0.8506721 ]],

       [[-0.862745  , -0.40756297, -0.9852941 , ..., -0.512605  ,
         -0.65266097, -0.9502801 ],
        [-1.0903361 , -0.65266097, -0.9852941 , ..., -0.47759098,
         -0.37254897, -0.77521   ],
        [-1.352941  , -1.0728291 , -1.12535   , ..., -0.495098  ,
         -0.582633  , -0.77521   ],
        ...,
        [-1.8431373 , -1.8256302 , -1.7906162 , ..., -

### EfficientNet 모델(pre-trained) 가져오기

In [73]:
import torchvision.models as models
from torchvision.models import EfficientNet_B1_Weights

weights = EfficientNet_B1_Weights.IMAGENET1K_V1
transform = weights.transforms()

In [74]:
model = models.efficientnet_b1(weights=weights)

In [16]:
# import torch
# import torchvision.models as models

# model = models.efficientnet_b0(pretrained=True)
# # model.classifier[1] = torch.nn.Linear(1280, 2)

In [10]:
# from torchvision.models import efficientnet_b0


# model = efficientnet_b0()
# model

In [17]:
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

out_features가 1000으로 되어있다. 우리는 개와 고양이, 총 2개로 분류할 예정이기 때문에 2로 바꿔주어야 한다.

In [75]:
model.classifier[1] = torch.nn.Linear(in_features=1280, out_features=2)

In [76]:
model.classifier[1]
# out_features 변경 완료

Linear(in_features=1280, out_features=2, bias=True)

In [58]:
from torchsummary import summary

summary(model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]             864
       BatchNorm2d-2           [-1, 32, 64, 64]              64
              SiLU-3           [-1, 32, 64, 64]               0
            Conv2d-4           [-1, 32, 64, 64]             288
       BatchNorm2d-5           [-1, 32, 64, 64]              64
              SiLU-6           [-1, 32, 64, 64]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12           [-1, 32, 64, 64]               0
           Conv2d-13           [-1, 16, 64, 64]             512
      BatchNorm2d-14           [-1, 16,

### 모델 학습 코드

In [77]:
def get_mean(metrics):
    return round(sum(metrics) / len(metrics), 4)

In [78]:
from collections import defaultdict
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score


def train_model(model):
    model.train()
    loss_list = []
    acc_list = []
    for x_train, y_train in tqdm(train_dataloader):
        x_train = x_train.to(device)
        y_train = y_train.to(device)

        outputs = model(x_train)
        loss = criterion(outputs, y_train)
        loss_list.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred = torch.argmax(outputs, dim=1)
        acc = ((y_train == pred).sum() / len(y_train)).item()
        acc_list.append(acc)
    return get_mean(loss_list), get_mean(acc_list)

In [79]:
def validate_model(model):
    model.eval()
    loss_list = []
    acc_list = []
    for x_val, y_val in tqdm(val_dataloader):
        x_val = x_val.to(device)
        y_val = y_val.to(device)
        with torch.no_grad():
            outputs = model(x_val)
            loss = criterion(outputs, y_val)
            loss_list.append(loss.item())

            pred = torch.argmax(outputs, dim=1)
            acc = ((y_val == pred).sum() / len(y_val)).item()
            acc_list.append(acc)
    return get_mean(loss_list), get_mean(acc_list)

In [80]:
from collections import defaultdict


def train_validate_model(model):
    logs = defaultdict(list)
    for epoch in range(epochs):
        train_loss, train_acc = train_model(model)
        val_loss, val_acc = validate_model(model)
        logs["train_loss"].append(train_loss)
        logs["train_acc"].append(train_acc)
        logs["val_loss"].append(val_loss)
        logs["val_acc"].append(val_acc)
        print(f"epoch {epoch + 1} train - loss: {train_loss} acc: {train_acc} val - loss: {val_loss} acc: {val_acc}")
    return logs

In [81]:
from matplotlib import pyplot as plt

def plot_logs(logs):
    fig = plt.figure(figsize=(10, 4))

    ax0 = fig.add_subplot(1, 2, 1)
    ax1 = fig.add_subplot(1, 2, 2)
    ax0.plot(logs["train_loss"], label="train")
    ax0.plot(logs["val_loss"], label="val")
    ax0.legend()
    ax0.set_title("loss")

    ax1.plot(logs["train_acc"], label="train")
    ax1.plot(logs["val_acc"], label="val")
    ax1.legend()
    ax1.set_title("accuracy")
    plt.legend()

## 1. FC layer만 학습

### 하이퍼 파라미터 설정

In [82]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()
epochs = 3

In [83]:
# fc layer를 제외한 다른 layer들은 학습되지 않도록 얼림
for param in model.parameters():
  param.requires_grad = False

# fc를 우리 데이터셋에 맞게 교체
model.classifier[1] = torch.nn.Linear(in_features=1280, out_features=2)
model = model.to(device)

In [84]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [85]:
model = model.to(device)

In [86]:
model.classifier[1]

Linear(in_features=1280, out_features=2, bias=True)

In [ ]:
logs = train_validate_model(model)

100%|██████████| 16/16 [05:17<00:00, 19.82s/it]


epoch 1 train - loss: 0.7561 acc: 0.3679 val - loss: 0.7468 acc: 0.3678


100%|██████████| 16/16 [05:04<00:00, 19.01s/it]


epoch 2 train - loss: 0.7537 acc: 0.3874 val - loss: 0.7474 acc: 0.3639


 67%|██████▋   | 42/63 [16:47<08:15, 23.60s/it]

In [ ]:
plot_logs(logs)